In [1]:
puts `pwd`
puts `ls -l`
puts `head rawdata/disease-therapeuticarea.csv`

/home/osboxes/CODE/SIMPATHIC/SKG_Mapping/radboud
total 32
drwxrwxr-x 2 osboxes osboxes 4096 May 19 10:28 mappings
-rw-rw-r-- 1 osboxes osboxes  112 Feb 28 19:02 notes
-rw-rw-r-- 1 osboxes osboxes 6827 May 19 10:30 radboud disease gene.ipynb
-rw-rw-r-- 1 osboxes osboxes 6496 May 19 11:12 radboud disease therapeutic area.ipynb
drwxrwxr-x 2 osboxes osboxes 4096 May 19 10:29 rawdata
drwxrwxr-x 2 osboxes osboxes 4096 May 19 10:26 scripts to copy into Jupyter
"source","source_type","target","target_type"
"DOID_7551","disease","OTAR_0000017","therapeutic_area"
"DOID_7551","disease","EFO_0005741","therapeutic_area"
"DOID_7551","disease","EFO_0009690","therapeutic_area"
"HP_0000031","disease","EFO_0000651","therapeutic_area"
"HP_0000857","disease","EFO_0000651","therapeutic_area"
"HP_0000857","disease","MONDO_0024297","therapeutic_area"
"HP_0000857","disease","EFO_0001379","therapeutic_area"
"HP_0000857","disease","EFO_0009605","therapeutic_area"
"HP_0000857","disease","EFO_0010282","therapeuti

In [6]:
require 'csv'
puts `pwd`

targets = {}

CSV.foreach('./rawdata/disease-therapeuticarea.csv', headers: true) do |row|
  next if row.size < 3

  source = row[0]
  target = row[2] || ''

  target = row[0].match(/([^_]+)/)[1]
  targets[target] = 1

end
puts "Targets"
puts targets.keys

/home/osboxes/CODE/SIMPATHIC/SKG_Mapping/radboud
Targets
DOID
HP
MONDO
EFO
OBI
NCIT
Orphanet
OTAR


# Process file

Need to process both the source and the target

Source and target need pre-processing to make URLs that are correct for the metadata_functions routines.


In [ ]:
require_relative '../Lookups/metadata_functions'
require 'csv'

# lookup = NCBO.new
f = File.open('./mappings/disease-therapeuticarea-mappings.csv', 'w')
e = File.open('./mappings/disease-therapeuticarea-errors.txt', 'w')
f.sync = true # Ensure immediate writes
e.sync = true # Ensure immediate writes
f.write "radboudsource,sourceguid,sourcelabel,radboudtarget,targetguid,targetlabel\n"

seen = {}
CSV.foreach('./rawdata/disease-therapeuticarea.csv', headers: true) do |row|
  next if row.size < 3

  # PROCESS SOURCE COLUMN #######################################
  radboudsource = row[0]
  ontology = radboudsource.match(/([^_]+)_/)[1]

  if ontology == "EFO"
    sourceurl = "http://www.ebi.ac.uk/efo/#{radboudsource}"
  elsif ontology == "OTAR"
    sourceurl = "http://www.ebi.ac.uk/efo/#{radboudsource}"
  elsif ontology == "DOID"
    sourceurl = "https://api.disease-ontology.org/v1/terms/#{radboudsource}"
  elsif ontology == "Orphanet"
    sourceurl = "http://www.orpha.net/ORDO/#{radboudsource}"
  else
    sourceurl = "http://purl.obolibrary.org/obo/#{radboudsource}"
  end

  sourcetitle = ontology_annotations(uri: sourceurl)
  # lookup.lookup_title_by_uri(term_uri: url, ontology: ontology)
  if sourcetitle
    warn "#{radboudsource}, #{sourceurl}, #{sourcetitle}"
  else # this shoujld never happen because the raw curie is returned if it fails
    warn "No match for #{source}, #{url}"
    e.write "No match for #{source}, #{url}"
    next
  end
  
# PROCESS TARGET COLUMN #######################################
  radboudtarget = row[2]
  ontology = radboudtarget.match(/([^_]+)_/)[1]

  if ontology == "EFO"
    targeturl = "http://www.ebi.ac.uk/efo/#{radboudtarget}"
  elsif ontology == "OTAR"
    targeturl = "http://www.ebi.ac.uk/efo/#{radboudtarget}"
  elsif ontology == "DOID"
    targeturl = "https://api.disease-ontology.org/v1/terms/#{radboudtarget}"
  elsif ontology == "Orphanet"
    targeturl = "http://www.orpha.net/ORDO/#{radboudtarget}"
  else
    targeturl = "http://purl.obolibrary.org/obo/#{radboudtarget}"
  end

  targettitle = ontology_annotations(uri: targeturl)
  # lookup.lookup_title_by_uri(term_uri: url, ontology: ontology)
  if targettitle
    warn "#{radboudtarget}, #{targeturl}, #{targettitle}"
  else # this shoujld never happen because the raw curie is returned if it fails
    warn "No match for #{radboudtarget}, #{targeturl}"
    e.write "No match for #{radboudtarget}, #{targeturl}"
    next  
  end
  
  

  f.write CSV.generate_line([radboudsource, sourceurl, sourcetitle, radboudtarget, targeturl, targettitle])

end

f.close
e.close

Final URL list ["https://api.disease-ontology.org/v1/terms/DOID_7551"]

processing https://api.disease-ontology.org/v1/terms/DOID_7551
DOID
term: gonorrhea
DOID_7551, https://api.disease-ontology.org/v1/terms/DOID_7551, gonorrhea
Final URL list ["http://www.ebi.ac.uk/efo/OTAR_0000017"]

processing http://www.ebi.ac.uk/efo/OTAR_0000017
just a URL
term: OTAR_0000017
OTAR_0000017, http://www.ebi.ac.uk/efo/OTAR_0000017, OTAR_0000017
Final URL list ["https://api.disease-ontology.org/v1/terms/DOID_7551"]

processing https://api.disease-ontology.org/v1/terms/DOID_7551
DOID
term: gonorrhea
DOID_7551, https://api.disease-ontology.org/v1/terms/DOID_7551, gonorrhea
Final URL list ["http://www.ebi.ac.uk/efo/EFO_0005741"]

processing http://www.ebi.ac.uk/efo/EFO_0005741
EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/efo/terms/http%253A%252F%252Fwww.ebi.ac.uk%252Fefo%252FEFO_0005741
term: infectious disease
EFO_0005741, http://www.ebi.ac.uk/efo/EFO_0005741, infectious disease
Final URL l

term: hepatocellular clear cell carcinoma
MONDO_0003243, http://purl.obolibrary.org/obo/MONDO_0003243, hepatocellular clear cell carcinoma
Final URL list ["http://purl.obolibrary.org/obo/MONDO_0045024"]

processing http://purl.obolibrary.org/obo/MONDO_0045024
EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/mondo/terms/http%253A%252F%252Fpurl.obolibrary.org%252Fobo%252FMONDO_0045024
term: cancer or benign tumor
MONDO_0045024, http://purl.obolibrary.org/obo/MONDO_0045024, cancer or benign tumor
Final URL list ["http://purl.obolibrary.org/obo/MONDO_0030006"]

processing http://purl.obolibrary.org/obo/MONDO_0030006
EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/mondo/terms/http%253A%252F%252Fpurl.obolibrary.org%252Fobo%252FMONDO_0030006
term: combined oxidative phosphorylation deficiency 40
MONDO_0030006, http://purl.obolibrary.org/obo/MONDO_0030006, combined oxidative phosphorylation deficiency 40
Final URL list ["http://purl.obolibrary.org/obo/MONDO_0024297"]

proc

MONDO_0009213, http://purl.obolibrary.org/obo/MONDO_0009213, Fanconi anemia complementation group C
Final URL list ["http://www.ebi.ac.uk/efo/EFO_0005803"]

processing http://www.ebi.ac.uk/efo/EFO_0005803
EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/efo/terms/http%253A%252F%252Fwww.ebi.ac.uk%252Fefo%252FEFO_0005803
term: hematologic disease
EFO_0005803, http://www.ebi.ac.uk/efo/EFO_0005803, hematologic disease
Final URL list ["http://purl.obolibrary.org/obo/MONDO_0009213"]

processing http://purl.obolibrary.org/obo/MONDO_0009213
EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/mondo/terms/http%253A%252F%252Fpurl.obolibrary.org%252Fobo%252FMONDO_0009213
term: Fanconi anemia complementation group C
MONDO_0009213, http://purl.obolibrary.org/obo/MONDO_0009213, Fanconi anemia complementation group C
Final URL list ["http://www.ebi.ac.uk/efo/EFO_0010285"]

processing http://www.ebi.ac.uk/efo/EFO_0010285
EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/efo/te

EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/efo/terms/http%253A%252F%252Fwww.ebi.ac.uk%252Fefo%252FEFO_0000781
term: Pectobacterium carotovorum infection
EFO_0000781, http://www.ebi.ac.uk/efo/EFO_0000781, Pectobacterium carotovorum infection
Final URL list ["http://www.ebi.ac.uk/efo/EFO_0005741"]

processing http://www.ebi.ac.uk/efo/EFO_0005741
EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/efo/terms/http%253A%252F%252Fwww.ebi.ac.uk%252Fefo%252FEFO_0005741
term: infectious disease
EFO_0005741, http://www.ebi.ac.uk/efo/EFO_0005741, infectious disease
Final URL list ["http://purl.obolibrary.org/obo/HP_0001310"]

processing http://purl.obolibrary.org/obo/HP_0001310
EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/hp/terms/http%253A%252F%252Fpurl.obolibrary.org%252Fobo%252FHP_0001310
term: Dysmetria
HP_0001310, http://purl.obolibrary.org/obo/HP_0001310, Dysmetria
Final URL list ["http://www.ebi.ac.uk/efo/EFO_0000651"]

processing http://www.ebi.ac.uk/ef

EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/mondo/terms/http%253A%252F%252Fpurl.obolibrary.org%252Fobo%252FMONDO_0012806
term: ectodermal dysplasia and immunodeficiency 2
MONDO_0012806, http://purl.obolibrary.org/obo/MONDO_0012806, ectodermal dysplasia and immunodeficiency 2
Final URL list ["http://purl.obolibrary.org/obo/MONDO_0045024"]

processing http://purl.obolibrary.org/obo/MONDO_0045024
EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/mondo/terms/http%253A%252F%252Fpurl.obolibrary.org%252Fobo%252FMONDO_0045024
term: cancer or benign tumor
MONDO_0045024, http://purl.obolibrary.org/obo/MONDO_0045024, cancer or benign tumor
Final URL list ["http://purl.obolibrary.org/obo/MONDO_0012806"]

processing http://purl.obolibrary.org/obo/MONDO_0012806
EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/mondo/terms/http%253A%252F%252Fpurl.obolibrary.org%252Fobo%252FMONDO_0012806
term: ectodermal dysplasia and immunodeficiency 2
MONDO_0012806, http://purl.oboli

...

term: benign neoplasm of pancreas
MONDO_0021470, http://purl.obolibrary.org/obo/MONDO_0021470, benign neoplasm of pancreas
Final URL list ["http://www.ebi.ac.uk/efo/EFO_0009605"]

processing http://www.ebi.ac.uk/efo/EFO_0009605
EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/efo/terms/http%253A%252F%252Fwww.ebi.ac.uk%252Fefo%252FEFO_0009605
term: pancreas disease
EFO_0009605, http://www.ebi.ac.uk/efo/EFO_0009605, pancreas disease
Final URL list ["http://purl.obolibrary.org/obo/MONDO_0021470"]

processing http://purl.obolibrary.org/obo/MONDO_0021470
EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/mondo/terms/http%253A%252F%252Fpurl.obolibrary.org%252Fobo%252FMONDO_0021470
term: benign neoplasm of pancreas
MONDO_0021470, http://purl.obolibrary.org/obo/MONDO_0021470, benign neoplasm of pancreas
Final URL list ["http://www.ebi.ac.uk/efo/EFO_0010282"]

processing http://www.ebi.ac.uk/efo/EFO_0010282
EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/efo/terms/h

term: ectrodactyly, ectodermal dysplasia, and cleft lip-palate syndrome 3
MONDO_0011428, http://purl.obolibrary.org/obo/MONDO_0011428, ectrodactyly, ectodermal dysplasia, and cleft lip-palate syndrome 3
Final URL list ["http://www.ebi.ac.uk/efo/EFO_0009690"]

processing http://www.ebi.ac.uk/efo/EFO_0009690
EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/efo/terms/http%253A%252F%252Fwww.ebi.ac.uk%252Fefo%252FEFO_0009690
term: urinary system disease
EFO_0009690, http://www.ebi.ac.uk/efo/EFO_0009690, urinary system disease
Final URL list ["http://purl.obolibrary.org/obo/MONDO_0011428"]

processing http://purl.obolibrary.org/obo/MONDO_0011428
EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/mondo/terms/http%253A%252F%252Fpurl.obolibrary.org%252Fobo%252FMONDO_0011428
term: ectrodactyly, ectodermal dysplasia, and cleft lip-palate syndrome 3
MONDO_0011428, http://purl.obolibrary.org/obo/MONDO_0011428, ectrodactyly, ectodermal dysplasia, and cleft lip-palate syndrome 3
Fin

term: immunodeficiency 23
MONDO_0014353, http://purl.obolibrary.org/obo/MONDO_0014353, immunodeficiency 23
Final URL list ["http://www.ebi.ac.uk/efo/EFO_0000618"]

processing http://www.ebi.ac.uk/efo/EFO_0000618
EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/efo/terms/http%253A%252F%252Fwww.ebi.ac.uk%252Fefo%252FEFO_0000618
term: nervous system disease
EFO_0000618, http://www.ebi.ac.uk/efo/EFO_0000618, nervous system disease
Final URL list ["http://purl.obolibrary.org/obo/MONDO_0014353"]

processing http://purl.obolibrary.org/obo/MONDO_0014353
EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/mondo/terms/http%253A%252F%252Fpurl.obolibrary.org%252Fobo%252FMONDO_0014353
term: immunodeficiency 23
MONDO_0014353, http://purl.obolibrary.org/obo/MONDO_0014353, immunodeficiency 23
Final URL list ["http://www.ebi.ac.uk/efo/OTAR_0000018"]

processing http://www.ebi.ac.uk/efo/OTAR_0000018
just a URL
term: OTAR_0000018
OTAR_0000018, http://www.ebi.ac.uk/efo/OTAR_0000018, OTAR_

Final URL list ["http://www.ebi.ac.uk/efo/EFO_0000618"]

processing http://www.ebi.ac.uk/efo/EFO_0000618
EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/efo/terms/http%253A%252F%252Fwww.ebi.ac.uk%252Fefo%252FEFO_0000618
term: nervous system disease
EFO_0000618, http://www.ebi.ac.uk/efo/EFO_0000618, nervous system disease
Final URL list ["http://purl.obolibrary.org/obo/MONDO_0054860"]

processing http://purl.obolibrary.org/obo/MONDO_0054860
EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/mondo/terms/http%253A%252F%252Fpurl.obolibrary.org%252Fobo%252FMONDO_0054860
term: hearing loss, autosomal recessive 110
MONDO_0054860, http://purl.obolibrary.org/obo/MONDO_0054860, hearing loss, autosomal recessive 110
Final URL list ["http://www.ebi.ac.uk/efo/OTAR_0000018"]

processing http://www.ebi.ac.uk/efo/OTAR_0000018
just a URL
term: OTAR_0000018
OTAR_0000018, http://www.ebi.ac.uk/efo/OTAR_0000018, OTAR_0000018
Final URL list ["http://purl.obolibrary.org/obo/MONDO_0100194"]

processing http://www.ebi.ac.uk/efo/EFO_0000651
EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/efo/terms/http%253A%252F%252Fwww.ebi.ac.uk%252Fefo%252FEFO_0000651
term: phenotype
EFO_0000651, http://www.ebi.ac.uk/efo/EFO_0000651, phenotype
Final URL list ["http://purl.obolibrary.org/obo/HP_0002290"]

processing http://purl.obolibrary.org/obo/HP_0002290
EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/hp/terms/http%253A%252F%252Fpurl.obolibrary.org%252Fobo%252FHP_0002290
term: Poliosis
HP_0002290, http://purl.obolibrary.org/obo/HP_0002290, Poliosis
Final URL list ["http://www.ebi.ac.uk/efo/EFO_0000651"]

processing http://www.ebi.ac.uk/efo/EFO_0000651
EBI
final URL is https://www.ebi.ac.uk/ols4/api/ontologies/efo/terms/http%253A%252F%252Fwww.ebi.ac.uk%252Fefo%252FEFO_0000651
term: phenotype
EFO_0000651, http://www.ebi.ac.uk/efo/EFO_0000651, phenotype
Final URL list ["http://purl.obolibrary.org/obo/HP_0006467"]

processing http://purl.obolibrary.org/obo/HP_0006467
EB